In [ ]:
# CÉLULA 1: Instalação e Configuração do Ambiente
#--------------------------------------------------
# Antes de começar, esta célula instala as bibliotecas necessárias para a análise.
!pip install pvlib pandas geopandas matplotlib

# Importação das bibliotecas que serão utilizadas no notebook
import pandas as pd
import pvlib
import matplotlib.pyplot as plt

print("Ambiente configurado com sucesso.")

In [ ]:
# CÉLULA 2: Carregamento dos Dados de Entrada
#--------------------------------------------------
# Carregamos o arquivo CSV para extrair os dados de um "lead" para a simulação.
try:
    df = pd.read_csv('leads_enriched_final.csv')
    # Selecionamos o primeiro lead do arquivo como exemplo para a nossa análise
    lead_exemplo = df.iloc[0]
    latitude = lead_exemplo['lat']
    longitude = lead_exemplo['lon']
    print(f"Dados carregados. Usando a localização de exemplo: Latitude={latitude}, Longitude={longitude}")
except FileNotFoundError:
    print("Arquivo 'leads_enriched_final.csv' não encontrado. Usando localização de exemplo (São Paulo, SP).")
    latitude, longitude = -23.55, -46.63

In [ ]:
# 1. Domínio: Caracterização do Potencial Solar e Geometria Astronômica
# CÉLULA 3: Referenciamento Geoespacial e Cálculo de Efemérides Solares
#--------------------------------------------------------------------------
from pvlib.location import Location

# 1.1. Referenciamento Geoespacial: Definição do local e fuso horário.
location = Location(latitude, longitude, tz='America/Sao_Paulo', altitude=760, name='Lead Exemplo')

# 1.2. Cálculo de Efemérides Solares: Determinação da posição solar para um ano inteiro.
# Criamos uma série temporal horária para o ano de 2023.
times = pd.date_range(start='2023-01-01', end='2024-01-01', freq='h', tz=location.tz)
solar_position = location.get_solarposition(times)

# Visualização dos resultados deste domínio
print("--- Domínio 1: Caracterização do Potencial Solar e Geometria Astronômica ---")
print("Posição solar calculada para o período:")
print(solar_position.head())

# Plotando a altitude solar ao longo de um dia de verão e um de inverno
plt.figure(figsize=(12, 6))
solar_position['apparent_elevation'].loc['2023-12-21'].plot(label='Solstício de Verão')
solar_position['apparent_elevation'].loc['2023-06-21'].plot(label='Solstício de Inverno')
plt.title('Altitude Solar Aparente ao Longo do Dia')
plt.ylabel('Elevação (graus)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 2. Domínio: Modelagem Radiométrica e Climatológica
# CÉLULA 4: Aquisição de Dados Meteorológicos e Modelagem de Transposição
#----------------------------------------------------------------------------

# 2.1. Aquisição de Séries Temporais Meteorológicas (TMY - Ano Meteorológico Típico do PVGIS)
# O TMY fornece um ano de dados horários representativos para o local.
weather, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude, longitude, map_variables=True)

# 2.2. Modelagem de Transposição: Calculando a irradiância no plano dos painéis.
# Assumimos uma inclinação (tilt) ótima para a latitude e azimute voltado para o norte geográfico (180°).
surface_tilt = latitude * -1
surface_azimuth = 180
poa_irradiance = pvlib.irradiance.get_total_irradiance(
    surface_tilt=surface_tilt,
    surface_azimuth=surface_azimuth,
    solar_zenith=solar_position['apparent_zenith'],
    solar_azimuth=solar_position['azimuth'],
    dni=weather['dni'],
    ghi=weather['ghi'],
    dhi=weather['dhi'],
    albedo=0.2  # Albedo típico para solo/vegetação
)

# Combinar a irradiância POA (Plane of Array) em um único DataFrame
weather['poa_global'] = poa_irradiance['poa_global']

# Visualização dos resultados
print("
--- Domínio 2: Modelagem Radiométrica e Climatológica ---")
print("Dados meteorológicos e de irradiância (primeiras 5 horas do ano):")
print(weather[['ghi', 'dni', 'dhi', 'poa_global', 'temp_air']].head())

# Plotando a irradiância em um dia de céu claro
plt.figure(figsize=(12, 6))
weather[['ghi', 'dhi', 'dni', 'poa_global']].loc['2023-01-05'].plot()
plt.title('Componentes da Irradiância em um Dia de Janeiro')
plt.ylabel('Irradiância (W/m²)')
plt.grid(True)
plt.show()

In [ ]:
# 3. Domínio: Modelagem da Geração e Conversão de Energia do Sistema FV
# CÉLULA 5: Parametrização dos Componentes e Simulação do Balanço Energético
#---------------------------------------------------------------------------------
from pvlib.pvsystem import PVSystem, FixedMount
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.modelchain import ModelChain

# 3.1. Parametrização de Componentes: Módulo e Inversor
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('CECInverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temp_params = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

# Definindo a montagem e o sistema FV (para este exemplo, um sistema pequeno)
mount = FixedMount(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth)
system = PVSystem(mounts=[mount], module_parameters=module, inverter_parameters=inverter,
                  temperature_model_parameters=temp_params)

# 3.2. Simulação do Balanço Energético com a ModelChain
mc = ModelChain(system, location)
mc.run_model(weather)

# Visualização dos resultados
print("
--- Domínio 3: Modelagem da Geração e Conversão de Energia do Sistema FV ---")
print("Resultados da simulação da ModelChain (Potência AC em Watts):")
print(mc.results.ac.head())

# Plotando a geração de energia AC em uma semana
plt.figure(figsize=(12, 6))
mc.results.ac.loc['2023-01-01':'2023-01-07'].plot()
plt.title('Produção de Energia AC Simulada (Primeira Semana do Ano)')
plt.ylabel('Potência (W)')
plt.grid(True)
plt.show()

In [ ]:
# 4. Domínio: Análise de Desempenho, Perdas e Indicadores de Viabilidade
# CÉLULA 6: Quantificação de Perdas e Cálculo de KPIs
#--------------------------------------------------------

# 4.1. Quantificação de Perdas (Exemplo: Perdas térmicas)
# A ModelChain já calcula a temperatura da célula, que é uma das principais fontes de perdas.
# Uma análise mais aprofundada poderia incluir modelos de soiling, degradação, etc.

# 4.2. Cálculo de Indicadores de Performance (KPIs)
# Potência Nominal DC do sistema (em kW)
p_dc_nominal = module['Impo'] * module['Vmpo'] / 1000  # Potência de 1 módulo

# Energia Anual AC Gerada (em kWh)
annual_ac_energy = mc.results.ac.sum() / 1000

# Irradiação Anual no Plano do Arranjo (POA) (em kWh/m²)
annual_poa = weather['poa_global'].sum() / 1000

# Performance Ratio (PR)
# Yf (Final Yield) = Energia AC / Potência Nominal DC
# Yr (Reference Yield) = Irradiação POA / Irradiância de Referência (1 kW/m²)
Yf = annual_ac_energy / p_dc_nominal
Yr = annual_poa / 1  # 1 kW/m² é a irradiação em STC
performance_ratio = (Yf / Yr) * 100

# Capacity Factor (CF)
hours_in_year = 8760
capacity_factor = (annual_ac_energy / (p_dc_nominal * hours_in_year)) * 100

# Apresentação dos KPIs
print("
--- Domínio 4: Análise de Desempenho, Perdas e Indicadores de Viabilidade ---")
print("
Resultados Anuais e KPIs de Performance:")
print("--------------------------------------------------")
print(f"Energia AC Anual Gerada...: {annual_ac_energy:.2f} kWh")
print(f"Irradiação Anual no Plano...: {annual_poa:.2f} kWh/m²")
print(f"Performance Ratio (PR)......: {performance_ratio:.2f} %")
print(f"Capacity Factor (CF)........: {capacity_factor:.2f} %")
print("--------------------------------------------------")